In [29]:
# from google.colab import drive
# drive.mount('/content/drive')

In [30]:
# %cd /content/drive/MyDrive/VQA_Med

In [2]:
path = '/home/arunava'
#****************************To be Change to reproduce ou result*********************************************

# location of the data and where to store iamge feature image
path_output_chd = path+'/VQA_Med'    

input_vqa_train = 'train_dataset_pickle/train_dataset_df.pkl'
input_vqa_valid ='valid_dataset_pickle/valid_dataset_df.pkl'

img_feat_train = 'train_dataset_pickle/train-image-feature.pickle'
img_feat_valid ='valid_dataset_pickle/valid-image-feature.pickle'

input_test = 'test_dataset_pickle/C1_test_dataset_df.pkl'
img_feat_test = 'test_dataset_pickle/test-image-feature.pickle'

In [3]:
import json
with open(path+'/VQA_Med/answer_classes_2019_2020.json', 'r') as j:
        answer_classes = json.load(j)


l = len(answer_classes) 

print(l)

457


In [4]:
import easydict
opt = easydict.EasyDict({
        "SEED":97,
        "BATCH_SIZE": 64,
        "VAL_BATCH_SIZE": 64,
        "NUM_OUTPUT_UNITS": l,
        "MAX_QUESTION_LEN": 17,
        "IMAGE_CHANNEL": 1472,
        "INIT_LERARNING_RATE": 1e-4,
        "LAMNDA":0.0001,
        "MFB_FACTOR_NUM":5,
        "MFB_OUT_DIM":1000,
        "BERT_UNIT_NUM":768,
        "BERT_DROPOUT_RATIO":0.3,
        "MFB_DROPOUT_RATIO":0.1,
        "NUM_IMG_GLIMPSE":2,
        "NUM_QUESTION_GLIMPSE":2,
        "IMG_FEAT_SIZE":1,
        "IMG_INPUT_SIZE":224,
        "NUM_EPOCHS":100,
    })

In [5]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import re
import collections
import pickle
import os
from tqdm import tqdm
import json
import torch.nn as nn
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision
from PIL import Image
import torch.nn.functional as f
import random

/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [7]:
category_names = {'C1': 'Modality','C2': 'Plane','C3': 'Organ', 'C4': 'Abnormality','All': 'All dataset',}

In [8]:
path_change = path
seed_value = opt.SEED
print(seed_value) # 97
np.random.seed(seed_value) # return None
random.seed(seed_value) # return None
torch.manual_seed(seed_value) # return <torch._C.Generator object at 0x7f71cdf7a3d0>
torch.cuda.manual_seed(seed_value) # return None
torch.cuda.manual_seed_all(seed_value) # return None
torch.backends.cudnn.enabled = False 
''' backends.cudnn.enabled enables cudnn for some operations such as conv layers and RNNs, which can yield 
a significant speedup. The cudnn RNN implementation doesn’t support the backward operation during eval() 
and thus raises the error. You could disable cudnn for your workload (as already done) or try to call .train()
on the RNN module separately after using model.eval().'''
torch.backends.cudnn.benchmark = False
'''If your model does not change and your input sizes remain the same - then you may benefit from setting 
torch.backends.cudnn.benchmark = True.However, if your model changes: for instance, if you have layers that
are only "activated" when certain conditions are met, or you have layers inside a loop that can be iterated a 
different number of times, then setting torch.backends.cudnn.benchmark = True might stall your execution.'''
torch.backends.cudnn.deterministic = False
'''torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. 
Therefore, no, it will not guarantee that your training process is deterministic, since you're also using
torch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA.'''

97


"torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. \nTherefore, no, it will not guarantee that your training process is deterministic, since you're also using\ntorch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA."

In [35]:
#set path for the dataset

dataset_folder = '/VQA_Med_Dataset'
train = {
    '2018': '/train_2018',
    '2019': '/train_2019',
    '2020': '/train_2020',
}

val = {
    '2018': '/val_2018',
    '2019': '/val_2019',
    '2020': '/val_2020',
    '2021': '/val_2021'
}

test = {
    '2018': '/test_2018',
    '2019': '/test_2019',
    '2020': '/test_2020',
    '2021': '/val_2021'

}

# train_images_path = path_change+dataset_folder+train_images['2018']+'Train_images'


# valid_images_path = path_change+dataset_folder+val_images['2019']+'Val_images'


# test_images_path = path_change+ dataset_folder+test_images['2019']+'Test_images'

size = opt.IMG_INPUT_SIZE
size

224

In [36]:
#Extract image feature
class VGG19(nn.Module):
    def __init__(self):
        '''
             We remove all the fully-connected layers in the VGG19 network and the convolution outputs of different feature scales
                are concatenated after global average pooling and l2-norm to form a 1984-dimensional vector to represent the image
        '''
        super(VGG19,self).__init__()
        vgg_model = torchvision.models.vgg19(pretrained=True)	
        # resnet_model = torchvision.models.resnet(pretrained=True)

        self.Conv1 = nn.Sequential(*list(vgg_model.features.children())[0:4])
        self.Conv2 = nn.Sequential(*list(vgg_model.features.children())[4:9])
        self.Conv3 = nn.Sequential(*list(vgg_model.features.children())[9:16])
        self.Conv4 = nn.Sequential(*list(vgg_model.features.children())[16:23])
        self.Conv5 = nn.Sequential(*list(vgg_model.features.children())[23:30])
        self.Conv6 = nn.Sequential(*list(vgg_model.features.children())[30:36])

        self.avgpool = nn.Sequential(list(vgg_model.children())[1])

        # self.inc = nn.Linear(1984,2048)
    
    def forward(self,image):

        with torch.no_grad():
            out1 = self.Conv1(image)
            # print('out1 shape',out1.shape)
            out2 = self.Conv2(out1)
            out3 = self.Conv3(out2)
            out4 = self.Conv4(out3)          
            out5 = self.Conv5(out4)          # [N, 512, 14, 14]
            out6 = self.Conv6(out5) 
            out7 = self.avgpool(out6)

        #global average pooling
        out1 = out1.mean([2,3],keepdim=True)
        out2 = out2.mean([2,3],keepdim=True)
        out3 = out3.mean([2,3],keepdim=True)
        out4 = out4.mean([2,3],keepdim=True)
        out5 = out5.mean([2,3],keepdim=True)
        out6 = out6.mean([2,3],keepdim=True)
        out7 = out7.mean([2,3],keepdim=True)

       
         
        concat_features = torch.cat([out1,out2, out3, out4,out5,out6 ], 1) 
        # print('concat_features',concat_features.shape) # 1,1984,1,1
        # with torch.no_grad():
        #     concat_features = self.inc(torch.squeeze(concat_features))
        
        # concat_features = torch.unsqueeze(concat_features, 0)
        # concat_features = torch.unsqueeze(concat_features, 2)
        # concat_features = torch.unsqueeze(concat_features, 3)

        #l2-normalized feature vector
        l2_norm = concat_features.norm(p=2, dim=1, keepdim=True).detach() 
        concat_features = concat_features.div(l2_norm)               # l2-normalized feature vector
       

        batch_size = concat_features.shape[0]
        embedding_dim_size = concat_features.shape[1]
        image_feature = concat_features.view(batch_size, embedding_dim_size, -1).squeeze(0) # [N, 1984, 1]

        # print('image_feature shape',image_feature.shape)

        return image_feature

In [37]:
transform = {
    phase: transforms.Compose([transforms.RandomResizedCrop(size), 
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))]) 

    for phase in ['train', 'valid']}

test_transform = transforms.Compose([transforms.RandomResizedCrop(size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])

In [38]:
vgg19_model = VGG19().to(device)

In [39]:
! pip install torchinfo

In [40]:
from torchinfo import summary
print(summary(vgg19_model))

Layer (type:depth-idx)                   Param #
VGG19                                    --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       1,792
│    └─ReLU: 2-2                         --
│    └─Conv2d: 2-3                       36,928
│    └─ReLU: 2-4                         --
├─Sequential: 1-2                        --
│    └─MaxPool2d: 2-5                    --
│    └─Conv2d: 2-6                       73,856
│    └─ReLU: 2-7                         --
│    └─Conv2d: 2-8                       147,584
│    └─ReLU: 2-9                         --
├─Sequential: 1-3                        --
│    └─MaxPool2d: 2-10                   --
│    └─Conv2d: 2-11                      295,168
│    └─ReLU: 2-12                        --
│    └─Conv2d: 2-13                      590,080
│    └─ReLU: 2-14                        --
│    └─Conv2d: 2-15                      590,080
│    └─ReLU: 2-16                        --
├─Sequential: 1-4                       

In [44]:
#read a txt train file  and structure it in a dataframe
# def get_category_file_train(category_name, category_path ,images_path, transform=None):
    
#     data = []
    
#     dict_data = { 'image_id':[],
#                     'image_path':[],
#                 'Question':[],
#                 'Answer':[],
#               }
#     image_feat  = { }
#     with open(category_path) as f:
#         lines = f.readlines()
#     cnt=0
#     for element in lines:
#         pd_element = element.split('|')
#         dict_data['image_id'].append(pd_element[0])
#         dict_data['Question'].append(re.sub(r'\s+', ' ',pd_element[1]).strip())
#         dict_data['Answer'].append(pd_element[2].strip("\n")) 
        
#         image = Image.open(images_path+pd_element[0]+'.jpg').convert('RGB')
#         # print('images',type(image))
#         if transform:
#             image = transform(image)

#         dict_data['image_path'].append(images_path+pd_element[0]+'.jpg')
#         image_feature = vgg19_model(image[None,...].to(device))
#         # print('image_feature shape',image_feature.shape)
#         image_feat[pd_element[0]] = image_feature.cpu().numpy()

#         # cnt = cnt+1
#         # if (cnt > 10): break
    
#     df_data = pd.DataFrame(dict_data, columns = ['image_id',
#                                                     'image_path',
#                                                  'Question',
#                                                  'Answer',
#                                                  ])


#     # answer_freq = count_answer_freq(df_data)
    
#     # classes = get_most_frequent_classes(answer_freq, threshold=5)
    
#     # df_data['labels'] = df_data['Answer'].apply(lambda x : classes[x] if x in classes else classes['UNKNOWN'])
    
#     return df_data, image_feat


In [29]:
dict_data = { 'image_id':[],
                'image_path':[],
                'Question':[],
                'Answer':[],
            }
image_feat  = { }

In [46]:
# import csv
# file1 = open('/content/drive/MyDrive/VQA_Med_Dataset/train_2018/All_QA_Pairs_train.txt', 'a')
# with open('/content/drive/MyDrive/VQA_Med_Dataset/train_2018/Train-QA.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
#     spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
#     for row in spamreader:
#         row.pop(0)
#         r = '|'.join(row)
#         r =r+"\n"
#         file1.write(r)

#     file1.close()

In [47]:
# import csv
# file1 = open('/home/arunava/VQA_Med_Dataset/val_2018/All_QA_Pairs_train.txt', 'a')
# with open('/home/arunava/VQA_Med_Dataset/val_2018/ValQA.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
#     spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
#     for row in spamreader:
#         row.pop(0)
#         r = '|'.join(row)
#         r =r+"\n"
#         file1.write(r)

#     file1.close()

In [23]:
keys = ['2018','2019','2020','2021']

key = '2020'

# for key in keys:
train_images_path = path_change+dataset_folder+train[key]+'/Train_images/'
train_questions_path = path_change+dataset_folder+train[key]+'/All_QA_Pairs_train.txt'

print(train_images_path)
print(train_questions_path)

/home/arunava/VQA_Med_Dataset/train_2020/Train_images/
/home/arunava/VQA_Med_Dataset/train_2020/All_QA_Pairs_train.txt


In [49]:
# train_images_path_2019 = path_change+dataset_folder+train['2019']+'/Train_images/'
# train_images_path_2020 = path_change+dataset_folder+train['2020']+'/Train_images/'
# train_questions_path_2019 = path_change+dataset_folder+train['2019']+'/All_QA_Pairs_train.txt'
# train_questions_path_2020 = path_change+dataset_folder+train['2020']+'/All_QA_Pairs_train.txt'


# with open(train_questions_path_2020) as f:
#     lines = f.readlines()
#     for element in lines:
#         pd_element = element.split('|')
        


In [24]:
train_transform = transforms.Compose([transforms.RandomResizedCrop(size), 
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])

In [25]:
with open(train_questions_path) as f:
        lines = f.readlines()
        for element in lines:
            pd_element = element.split('|')
            dict_data['image_id'].append(pd_element[0])
            dict_data['Question'].append(re.sub(r'\s+', ' ',pd_element[1]).strip())
            dict_data['Answer'].append(pd_element[2].strip("\n")) 
            dict_data['image_path'].append(train_images_path+pd_element[0]+'.jpg')
            
            image = Image.open(train_images_path+pd_element[0]+'.jpg').convert('RGB')
            # print('images',type(image))
            if train_transform:
                image = train_transform(image)

            
            image_feature = vgg19_model(image[None,...].to(device))
            
            image_feat[pd_element[0]] = image_feature.cpu().numpy()

            
        
        

In [26]:
print(len(dict_data['image_id']))
print(len(dict_data['image_path']))
print(len(dict_data['Question']))
print(len(dict_data['Answer']))

16792
16792
16792
16792


In [27]:
print(dict_data['image_id'])
print(dict_data['image_path'])

['synpic41148', 'synpic43984', 'synpic38930', 'synpic52143', 'synpic20934', 'synpic19141', 'synpic25502', 'synpic38317', 'synpic22829', 'synpic39477', 'synpic54866', 'synpic60053', 'synpic28004', 'synpic51949', 'synpic45041', 'synpic13026', 'synpic52613', 'synpic21981', 'synpic53380', 'synpic21144', 'synpic20556', 'synpic39156', 'synpic41586', 'synpic36892', 'synpic20868', 'synpic27646', 'synpic21181', 'synpic19981', 'synpic54237', 'synpic33704', 'synpic43008', 'synpic23493', 'synpic33886', 'synpic26174', 'synpic27913', 'synpic55224', 'synpic58999', 'synpic47538', 'synpic45740', 'synpic21427', 'synpic22081', 'synpic58842', 'synpic38409', 'synpic36205', 'synpic24510', 'synpic46206', 'synpic26884', 'synpic59324', 'synpic57262', 'synpic28983', 'synpic24141', 'synpic48293', 'synpic41684', 'synpic36984', 'synpic16795', 'synpic48977', 'synpic60337', 'synpic58921', 'synpic60196', 'synpic51217', 'synpic30823', 'synpic49520', 'synpic45585', 'synpic45427', 'synpic43558', 'synpic56917', 'synpic19

In [22]:
# dict_data['image_path'].append('xyz')

In [28]:
df_data = pd.DataFrame(dict_data, columns = ['image_id','image_path','Question','Answer'])

In [29]:
df_data

image_id                                         image_path  \
0      synpic41148  /home/arunava/VQA_Med_Dataset/train_2019/Train...   
1      synpic43984  /home/arunava/VQA_Med_Dataset/train_2019/Train...   
2      synpic38930  /home/arunava/VQA_Med_Dataset/train_2019/Train...   
3      synpic52143  /home/arunava/VQA_Med_Dataset/train_2019/Train...   
4      synpic20934  /home/arunava/VQA_Med_Dataset/train_2019/Train...   
...            ...                                                ...   
16787  synpic40606  /home/arunava/VQA_Med_Dataset/train_2020/Train...   
16788  synpic53235  /home/arunava/VQA_Med_Dataset/train_2020/Train...   
16789  synpic33250  /home/arunava/VQA_Med_Dataset/train_2020/Train...   
16790  synpic27598  /home/arunava/VQA_Med_Dataset/train_2020/Train...   
16791  synpic29815  /home/arunava/VQA_Med_Dataset/train_2020/Train...   

                                                Question  \
0                            what kind of image is this?   
1                           is this a t1 weighted image?   
2      what type of imaging modality is used to acqui...   
3                             is this a noncontrast mri?   
4                   what type of image modality is this?   
...                                                  ...   
16787       what is most alarming about this ultrasound?   
16788     what is the primary abnormality in this image?   
16789                what is abnormal in the ultrasound?   
16790     what is the primary abnormality in this image?   
16791             what abnormality is seen in the image?   

                               Answer  
0                cta - ct angiography  
1                                  no  
2                     us - ultrasound  
3                                  no  
4                     xr - plain film  
...                               ...  
16787             parathyroid adenoma  
16788               pneumopericardium  
16789       testicular microlithiasis  
16790  dermoid cyst (cystic teratoma)  
16791                  osteochondroma  

[16792 rows x 4 columns]

In [30]:
df_data.shape

(16792, 4)

In [16]:
def count_answer_freq(df_data):
    '''
    count the frequence of each unique answer on the dataset
    '''
    all_answers = df_data['Answer'].values
    answer_freq_dict = defaultdict(int)
    for answer in all_answers:
        answer_freq_dict[answer] += 1
    answer_freq_dict_sort = dict(sorted(answer_freq_dict.items(), key=lambda x: x[1], reverse=True))

    return answer_freq_dict_sort

In [17]:
def get_most_frequent_classes(answer_freq_dict, threshold=1):
    print('threshold',threshold)
    final_classes = defaultdict(int)
    index = 0

    for answer,ans_freq in answer_freq_dict.items():

        if ans_freq >= threshold:
            final_classes[answer]= index         
        else:
            final_classes['UNKNOWN'] = index
            break
        index += 1  
    final_classes['UNKNOWN'] = index
    with open('answer_classes_2019_2020.json', 'w') as fp:
        json.dump(final_classes, fp)
    return final_classes

In [18]:
def parse_sentence(s):
    '''remove some character on the question'''
    s = s.replace(" - ", "-")
    s = s.lower()
    s = re.sub("\s\s+", " ", s)
    return s

In [19]:
answer_freq = count_answer_freq(df_data)

NameError: name 'df_data' is not defined

In [35]:
classes = get_most_frequent_classes(answer_freq, threshold=5)

threshold 5


In [36]:
df_data['labels'] = df_data['Answer'].apply(lambda x : classes[x] if x in classes else classes['UNKNOWN'])

In [37]:
df_data.to_csv('file1.csv')

In [38]:
df_data

image_id                                         image_path  \
0      synpic41148  /home/arunava/VQA_Med_Dataset/train_2019/Train...   
1      synpic43984  /home/arunava/VQA_Med_Dataset/train_2019/Train...   
2      synpic38930  /home/arunava/VQA_Med_Dataset/train_2019/Train...   
3      synpic52143  /home/arunava/VQA_Med_Dataset/train_2019/Train...   
4      synpic20934  /home/arunava/VQA_Med_Dataset/train_2019/Train...   
...            ...                                                ...   
16787  synpic40606  /home/arunava/VQA_Med_Dataset/train_2020/Train...   
16788  synpic53235  /home/arunava/VQA_Med_Dataset/train_2020/Train...   
16789  synpic33250  /home/arunava/VQA_Med_Dataset/train_2020/Train...   
16790  synpic27598  /home/arunava/VQA_Med_Dataset/train_2020/Train...   
16791  synpic29815  /home/arunava/VQA_Med_Dataset/train_2020/Train...   

                                                Question  \
0                            what kind of image is this?   
1                           is this a t1 weighted image?   
2      what type of imaging modality is used to acqui...   
3                             is this a noncontrast mri?   
4                   what type of image modality is this?   
...                                                  ...   
16787       what is most alarming about this ultrasound?   
16788     what is the primary abnormality in this image?   
16789                what is abnormal in the ultrasound?   
16790     what is the primary abnormality in this image?   
16791             what abnormality is seen in the image?   

                               Answer  labels  
0                cta - ct angiography      37  
1                                  no       2  
2                     us - ultrasound      15  
3                                  no       2  
4                     xr - plain film       5  
...                               ...     ...  
16787             parathyroid adenoma     118  
16788               pneumopericardium     179  
16789       testicular microlithiasis     167  
16790  dermoid cyst (cystic teratoma)     297  
16791                  osteochondroma      39  

[16792 rows x 5 columns]

In [30]:
pickle_name='train_dataset_df_2019_2020'
df_data.to_pickle(path_output_chd+'/train_dataset_pickle/'+pickle_name+'.pkl')

NameError: name 'df_data' is not defined

In [40]:
with open(path_output_chd+'/train_dataset_pickle/train-image-feature-2019-2020.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(image_feat, f, pickle.HIGHEST_PROTOCOL)

In [63]:
valid_dict_data = {
    'image_id':[],
    'image_path':[],
    'Question':[],
    'Answer':[]
}

image_feat_valid = {}

In [69]:
keys = ['2018','2019','2020','2021']

key = '2020'

# for key in keys:
val_images_path = path_change+dataset_folder+val[key]+'/Val_images/'
val_questions_path = path_change+dataset_folder+val[key]+'/All_QA_Pairs_val.txt'

print(val_images_path)
print(val_questions_path)

/home/arunava/VQA_Med_Dataset/val_2020/Val_images/
/home/arunava/VQA_Med_Dataset/val_2020/All_QA_Pairs_val.txt


In [65]:
valid_transform = transforms.Compose([transforms.RandomResizedCrop(size), 
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])

In [66]:
val_questions_path

'/home/arunava/VQA_Med_Dataset/val_2019/All_QA_Pairs_val.txt'

In [70]:
with open(val_questions_path) as f:
        lines = f.readlines()
        for element in lines:
            pd_element = element.split('|')
            valid_dict_data['image_id'].append(pd_element[0])
            valid_dict_data['Question'].append(re.sub(r'\s+', ' ',pd_element[1]).strip())
            valid_dict_data['Answer'].append(pd_element[2].strip("\n")) 
            valid_dict_data['image_path'].append(val_images_path+pd_element[0]+'.jpg')
            
            image = Image.open(val_images_path+pd_element[0]+'.jpg').convert('RGB')
            # print('images',type(image))
            if train_transform:
                image = valid_transform(image)

            
            image_feature = vgg19_model(image[None,...].to(device))
            
            image_feat_valid[pd_element[0]] = image_feature.cpu().numpy()

In [71]:
print(len(valid_dict_data['image_id']))
print(len(valid_dict_data['image_path']))
print(len(valid_dict_data['Question']))
print(len(valid_dict_data['Answer']))

2500
2500
2500
2500


In [72]:
valid_df_data = pd.DataFrame(valid_dict_data, columns = ['image_id','image_path','Question','Answer'])

In [73]:
valid_df_data['labels'] = valid_df_data['Answer'].apply(lambda x : classes[x] if x in classes else classes['UNKNOWN'])

In [75]:
pickle_name='valid_dataset_df_2019_2020'
valid_df_data.to_pickle(path_output_chd+'/valid_dataset_pickle/'+pickle_name+'.pkl')

In [76]:
with open(path_output_chd+'/valid_dataset_pickle/valid-image-feature-2019-2020.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(image_feat_valid, f, pickle.HIGHEST_PROTOCOL)

In [41]:
test_transform = transforms.Compose([transforms.RandomResizedCrop(size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])

In [55]:
test_dict_data = { 'image_id':[],
                'image_path':[],
                'Question':[],
                'Answer':[],
            }
image_feat_test  = { }

In [56]:
keys = ['2018','2019','2020','2021']

key = '2021'

# for key in keys:
test_images_path = path_change+dataset_folder+test[key]+'/Test_images/'
test_questions_path = path_change+dataset_folder+test[key]+'/All_QA_Pairs_test.txt'

print(test_images_path)
print(test_questions_path)

/home/arunava/VQA_Med_Dataset/val_2021/Test_images/
/home/arunava/VQA_Med_Dataset/val_2021/All_QA_Pairs_test.txt


In [57]:
with open(test_questions_path) as f:
        lines = f.readlines()
        for element in lines:
            pd_element = element.split('|')
            test_dict_data['image_id'].append(pd_element[0])
            test_dict_data['Question'].append(re.sub(r'\s+', ' ',pd_element[1]).strip())
            test_dict_data['Answer'].append(pd_element[2].strip("\n")) 
            test_dict_data['image_path'].append(test_images_path+pd_element[0]+'.jpg')
            
            image = Image.open(test_images_path+pd_element[0]+'.jpg').convert('RGB')
            # print('images',type(image))
            if test_transform:
                image = test_transform(image)

            
            image_feature = vgg19_model(image[None,...].to(device))
            
            image_feat_test[pd_element[0]] = image_feature.cpu().numpy()

In [58]:
print(len(test_dict_data['image_id']))
print(len(test_dict_data['image_path']))
print(len(test_dict_data['Question']))
print(len(test_dict_data['Answer']))

500
500
500
500


In [59]:
test_df_data = pd.DataFrame(test_dict_data, columns = ['image_id','image_path','Question','Answer'])

In [60]:
test_df_data['labels'] = test_df_data['Answer'].apply(lambda x : answer_classes[x] if x in answer_classes else answer_classes['UNKNOWN'])

In [61]:
pickle_name='test_dataset_df_2021'
test_df_data.to_pickle(path_output_chd+'/test_dataset_pickle/'+pickle_name+'.pkl')

In [62]:
with open(path_output_chd+'/test_dataset_pickle/test-image-feature-2021.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(image_feat_test, f, pickle.HIGHEST_PROTOCOL)

In [60]:
# make two dataframes 
# 2018 2019 2020
# 2019 2020

# prepare training dataset
# prepare validation dataset
# prepare testing dataset
